Был выбран CycleGAN.
Вначале реализуется своя архитектура на известной задаче яблоки/апельсины.

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
import torch.optim as optim
from time import time
import os
import errno
import torchvision.utils as vutils
from IPython import display
from matplotlib import pyplot as plt
import random
import time
import datetime
import sys
from torch.autograd import Variable
import itertools
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
from PIL import Image
import glob
from torch.utils.data import Dataset
from IPython.display import clear_output

In [ ]:
import pickle
from skimage import io

from tqdm import tqdm, tqdm_notebook
from pathlib import Path

from multiprocessing.pool import ThreadPool
import torch.nn as nn

from matplotlib import colors, pyplot as plt
%matplotlib inline


#Шаманство с датой

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip /content/drive/My\ Drive/Asian2.zip 

In [ ]:
 !unzip /content/drive/My\ Drive/European1.zip 

In [ ]:
def for_show(inp, mode = 'numpy'): # Для вывода картинок во время трейна
  if mode != 'numpy':
    inp = inp.numpy().transpose((1, 2, 0))
  else:
    inp = inp.transpose((1, 2, 0))
  mean = np.array((0.5,0.5,0.5))
  std = np.array((0.5,0.5,0.5))
  inp = std * inp + mean
  inp = np.clip(inp, 0, 1)
  return inp

In [ ]:
def imshow(inp, title=None, plt_ax=plt, default=False): # Функция вывода изображения

    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array((0.5,0.5,0.5))
    std = np.array((0.5,0.5,0.5))
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt_ax.imshow(inp)
    if title is not None:
        plt_ax.set_title(title)
    plt_ax.grid(False)

In [ ]:
train_as = Path('/content/Asian1')
train_eu = Path('/content/European1')

as_files = sorted(list(train_as.rglob('*.png')))
eu_files = sorted(list(train_eu.rglob('*.png')))

In [ ]:
asian = []
european = []
for filename in as_files:
  asian.append(str(filename))

for filename in eu_files:
  european.append(str(filename)) 

In [ ]:
class MakeDataset(Dataset): # Подготовка датасета

    def __init__(self, files, files1 = None, mode = None, mode_sample = None):
        super().__init__()
        # список файлов для загрузки
        self.files = sorted(files)
        self.len_ = len(self.files)
        self.mode = mode
        if self.mode == None:
            self.files1 = sorted(files1)
            self.len1_ = len(self.files1)
        self.mode_sample = mode_sample
                      
    def __len__(self):
        return self.len_
      
    def load_sample(self, file):
        image = Image.open(file)
        image.load()
        return image
  
    def __getitem__(self, index):
        # для преобразования изображений в тензоры PyTorch и нормализации входа
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
        ])
        x = self.load_sample(self.files[index % len(self.files)])
        x = self._prepare_sample(x)
        x = np.array(x / 255, dtype='float32')
        x = transform(x)
        if self.mode == None:
            if self.mode_sample == 'rand':
                y = self.load_sample(self.files1[random.randint(0, len(self.files1) - 1)])
            else:
                y = self.load_sample(self.files1[index % len(self.files)])
            y = self._prepare_sample(y)
            y = np.array(y / 255, dtype='float32')
            y = transform(y)

            return x, y
        else:
            return x
        
    def _prepare_sample(self, image):
        image = image.resize((64, 64))
        return np.array(image)

In [ ]:
def prov(file): # В датасете есть чёрно-белые картинки, поэтому отбрасываем их
  transform = transforms.ToTensor()
  image = Image.open(file)
  image.load()
  image = np.array(image.resize((64, 64)))
  image = np.array(image / 255, dtype='float32')
  image = transform(image)
  return image


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
print(device)

cuda


In [ ]:
def plot_gallery(images, h, w, n_row=3, n_col=6):
    """Helper function to plot a gallery of portraits"""
    plt.figure(figsize=(1.5 * n_col, 1.7 * n_row))
    plt.subplots_adjust(bottom=0, left=.01, right=.99, top=.90, hspace=.35)
    #images = np.array(images, dtype='float32')
    for i in range(n_row * n_col):
        plt.subplot(n_row, n_col, i + 1)
        inp = np.array(images[i])
        #inp.transpose((1, 2, 0))
        mean = np.array([0.5, 0.5, 0.5])
        std = np.array([0.5, 0.5, 0.5])
        inp = std * inp + mean
        try:
            plt.imshow(inp, cmap=plt.cm.gray, vmin=-1, vmax=1, interpolation='nearest')
            plt.xticks(())
            plt.yticks(())
        except:
            pass

In [ ]:
def To_tens(dataset):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    new_set = []
    for i in range(len(dataset)):
        x = np.array(dataset[i].transpose((0, 1, 2)) / 255, dtype='float32')
        new_set.append(transform(x))
    return torch.stack(new_set).permute(0, 2, 3, 1)

In [ ]:
from sklearn.model_selection import train_test_split

X_train = asian[0:1200]
X_val = asian[1201:1342]
Y_train = european[0:1200]
Y_val = european[1201:1342]
train_dataset = MakeDataset(X_train, Y_train)
test_dataset = MakeDataset(X_val, Y_val, mode = None)

In [ ]:
class encoder(nn.Module):
    def __init__(self):
        super(encoder, self).__init__()
        self.linear = nn.Linear(64*64*3, 2048)
        self.bn = nn.BatchNorm1d(2048)
        self.linear1 = nn.Linear(2048, 1024)
        self.bn1 = nn.BatchNorm1d(1024)
        self.linear2 = nn.Linear(1024, 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.mu = nn.Linear(256, 100)
        # self.bn3 = nn.BatchNorm1d(100)
        self.var = nn.Linear(256, 100)
        # self.bn4 = nn.BatchNorm1d(100)

    def gaussian_sampler(self, mu, logsigma):
            if self.training:
                std = logsigma.exp_()
                eps = Variable(std.data.new(std.size()).normal_())
                return eps.mul(std).add_(mu)
            else:
                return mu


    def forward(self, x):
        hidden = F.leaky_relu(self.bn(self.linear(x)))
        hidden1 = F.leaky_relu(self.bn1(self.linear1(hidden)))
        hidden2 = F.leaky_relu(self.bn2(self.linear2(hidden1)))
        mu = self.mu(hidden2)
        logsigma = self.var(hidden2)
        latent = self.gaussian_sampler(mu, logsigma)
        
        return mu, logsigma, latent

class decoder(nn.Module):
    def __init__(self):
        super(decoder, self).__init__()
        self.latent_to_hidden = nn.Linear(100, 256)
        self.bn3 = nn.BatchNorm1d(256)
        self.hidden_to_out = nn.Linear(256, 1024)
        self.bn4 = nn.BatchNorm1d(1024)
        self.hidden_to_out1 = nn.Linear(1024, 2048)
        self.bn5 = nn.BatchNorm1d(2048)
        self.hidden_to_out2 = nn.Linear(2048, 64*64*3)



    def forward(self, z):

          # z = self.gaussian_sampler(mu, logsigma)

          x = F.leaky_relu(self.bn3(self.latent_to_hidden(z)))
          x = F.leaky_relu(self.bn4(self.hidden_to_out(x)))
          x = F.leaky_relu(self.bn5(self.hidden_to_out1(x)))
          reconstruction = F.sigmoid(self.hidden_to_out2(x))
          
        
          return reconstruction


class VAE(nn.Module):

    def __init__(self, enc, dec):
            super().__init__()

            self.enc = enc
            self.dec = dec



    def forward(self, x):
            # encode
            mu, logsigma, latent = self.enc(x)
            # latent = self.gaussian_sampler(mu, logsigma)

            # decode
            predicted = self.dec(latent)
            return predicted, mu, logsigma


In [ ]:
def KL_divergence(mu, logsigma):
    """
    часть функции потерь, которая отвечает за "близость" латентных представлений разных людей
    """
    loss = -0.5 * torch.mean(1 + 2*logsigma - mu**2 - torch.exp(2*logsigma))
    return loss

def log_likelihood(reconstruction, x):
    """
    часть функции потерь, которая отвечает за качество реконструкции (как mse в обычном autoencoder)
    """
    loss = F.binary_cross_entropy
    return loss(reconstruction, x)

def loss_vae(x, mu, logsigma, reconstruction):
    return 0.5 *(KL_divergence(mu, logsigma) / (64*64)+log_likelihood(reconstruction, x))

In [ ]:
criterion = loss_vae
encoder = encoder()
decoder = decoder()
vae = VAE(encoder, decoder).to(device)


optim = torch.optim.Adam(vae.parameters(), lr = 0.0001)

In [ ]:
from tqdm import tqdm 

def train_vae(train, val, epochs, autoencoder, loss_fn, optim):
    train_loader = DataLoader(train, batch_size=8, shuffle=False)
    
    test_loader = DataLoader(val, batch_size=8, shuffle=False)
    trainl = []
    vall = []

    for epoch in tqdm(range(epochs)):
        # X_val = next(iter(val)
        recon = []
        train_loss = 0.0
        val_loss = 0.0
        # print('* Epoch %d/%d' % (epoch+1, epochs))
        
        autoencoder.train()
        
        # encoder.train()
        # decoder.train()
      

        for X_batch, Y_batch in train_loader:

          

            X_batch = X_batch.to(device)
            X_batch = X_batch.view(X_batch.size(0), 64*64*3)
            optim.zero_grad()
            # code = encoder(X_batch)
            # rec = decoder(code)
            rec, mu, log = autoencoder(X_batch)


            loss = loss_fn(torch.clamp(X_batch, 0, 1), mu, log, rec)
            loss.backward()
            optim.step()
          

            train_loss += loss.item()
        trainl.append(train_loss / len(train))

        torch.save(autoencoder.state_dict(), 'wheights.pt')



        # print('loss: %f' % train_loss)
        autoencoder.eval()
        # encoder.eval()
        # decoder.eval()
        for X_val, Y_batch in test_loader:
            X_val = X_val.to(device)
            X_val = X_val.view(X_val.size(0), 64*64*3)
            
            with torch.no_grad():
                # clear_output(wait=True)
                # latent_code = encoder(X_val)
                # reconstruction = decoder(latent_code)
                reconstruction, muu, logsigma =  autoencoder(X_val)
                # X_val = X_val.detach().to(device).numpy().reshape(X_val.size(0),45,45, 3)
                # reconstruction = reconstruction.detach().to(device).numpy().reshape(reconstruction.size(0), 45, 45, 3)
                # plot_gallery([X_val.astype(np.uint8), reconstruction.astype(np.uint8)], IMAGE_H, IMAGE_W, n_row=1, n_col=2)
                loss = loss_fn(torch.clamp(X_val, 0, 1), muu, logsigma, reconstruction)
                reconstruction = reconstruction.view(X_val.size(0), 3, 64, 64)
                # X_val = X_val.view(reconstruction.size(0), 3, 256, 256)

            recon.append(reconstruction)
            val_loss += loss.item()
        vall.append(val_loss / len(val))
        clear_output(wait=True)


        
        
          # plot_gallery([np.array(val[k][0]), np.array(recon[0][k].detach().cpu())], h = 256, w = 256, n_row=1, n_col=2)
        fig, ax = plt.subplots(nrows=1, ncols=2,figsize=(8, 8), \
                        sharey=True, sharex=True)
        for fig_x in ax.flatten():
              im_val = test_dataset[0][0]
              imshow(recon[1][0].detach().cpu(), plt_ax=fig_x)

        fig, ax = plt.subplots(nrows=1, ncols=2,figsize=(8, 8), \
                        sharey=True, sharex=True)
        for fig_x in ax.flatten():
              im_val = test_dataset[0][0]
              # imshow(im_val.reshape(256, 256, 3).detach().cpu(), plt_ax=fig_x)
              imshow(im_val.detach().cpu(), plt_ax=fig_x)
              # imshow(recon[1][0].detach().cpu(), plt_ax=fig_x)
        plt.show()

        plt.plot(range(len(trainl)), trainl, label ='train')
        plt.plot(range(len(vall)), vall, label ='val')
        plt.suptitle('%d / %d - loss_train: %f, loss_val: %f' % (epoch+1, epochs, trainl[epoch], vall[epoch]))
        # plt.suptitle('%d / %d - loss_train: %f, loss_val: %f' % (epoch+1, \
                                                                #  epochs, trainl[epoch], vall[epoch]))
        plt.legend()
        plt.show()

In [ ]:
train_vae(train_dataset, test_dataset, 20, vae, criterion, optim)

In [ ]:
import gc

gc.collect()

In [ ]:
criterion1 = loss_vae
encoder1 = encoder()
decoder1 = decoder()
vae1 = VAE(encoder1, decoder1).to(device)


optim1 = torch.optim.Adam(vae1.parameters(), lr = 0.0001)

In [ ]:
def train_vae1(train, val, epochs, autoencoder, loss_fn, optim):
    train_loader = DataLoader(train, batch_size=8, shuffle=False)
    
    test_loader = DataLoader(val, batch_size=8, shuffle=False)
    trainl = []
    vall = []

    for epoch in tqdm(range(epochs)):
        # X_val = next(iter(val)
        recon = []
        train_loss = 0.0
        val_loss = 0.0
        # print('* Epoch %d/%d' % (epoch+1, epochs))
        
        autoencoder.train()
        
        # encoder.train()
        # decoder.train()
      

        for X_batch, Y_batch in train_loader:

          

            Y_batch = Y_batch.to(device)
            Y_batch = Y_batch.view(Y_batch.size(0), 64*64*3)
            optim.zero_grad()
            # code = encoder(X_batch)
            # rec = decoder(code)
            rec, mu, log = autoencoder(Y_batch)


            loss = loss_fn(torch.clamp(Y_batch, 0, 1), mu, log, rec)
            loss.backward()
            optim.step()
          

            train_loss += loss.item()
        trainl.append(train_loss / len(train))

        torch.save(autoencoder.state_dict(), 'wheights1.pt')



        # print('loss: %f' % train_loss)
        autoencoder.eval()
        # encoder.eval()
        # decoder.eval()
        for X_val, Y_val in test_loader:
            Y_val = Y_val.to(device)
            Y_val = Y_val.view(Y_val.size(0), 64*64*3)
            
            with torch.no_grad():
                # clear_output(wait=True)
                # latent_code = encoder(X_val)
                # reconstruction = decoder(latent_code)
                reconstruction, muu, logsigma =  autoencoder(Y_val)
                # X_val = X_val.detach().to(device).numpy().reshape(X_val.size(0),45,45, 3)
                # reconstruction = reconstruction.detach().to(device).numpy().reshape(reconstruction.size(0), 45, 45, 3)
                # plot_gallery([X_val.astype(np.uint8), reconstruction.astype(np.uint8)], IMAGE_H, IMAGE_W, n_row=1, n_col=2)
                loss = loss_fn(torch.clamp(Y_val, 0, 1), muu, logsigma, reconstruction)
                reconstruction = reconstruction.view(Y_val.size(0), 3, 64, 64)
                # X_val = X_val.view(reconstruction.size(0), 3, 256, 256)

            recon.append(reconstruction)
            val_loss += loss.item()
        vall.append(val_loss / len(val))
        clear_output(wait=True)


        
        
          # plot_gallery([np.array(val[k][0]), np.array(recon[0][k].detach().cpu())], h = 256, w = 256, n_row=1, n_col=2)
        fig, ax = plt.subplots(nrows=1, ncols=2,figsize=(8, 8), \
                        sharey=True, sharex=True)
        for fig_x in ax.flatten():
              im_val = test_dataset[0][0]
              imshow(recon[1][1].detach().cpu(), plt_ax=fig_x)

        fig, ax = plt.subplots(nrows=1, ncols=2,figsize=(8, 8), \
                        sharey=True, sharex=True)
        for fig_x in ax.flatten():
              im_val = test_dataset[30][1]
              imshow(im_val.detach().cpu(), plt_ax=fig_x)

        plt.show()

        plt.plot(range(len(trainl)), trainl, label ='train')
        plt.plot(range(len(vall)), vall, label ='val')
        plt.suptitle('%d / %d - loss_train: %f, loss_val: %f' % (epoch+1, epochs, trainl[epoch], vall[epoch]))
        plt.legend()
        plt.show()

In [ ]:
train_vae1(train_dataset, test_dataset, 20, vae1, criterion1, optim1)